In [1]:
import pyodbc
import pandas as pd

server = 'jarbdatabase.database.windows.net' # For Azure SQL Database
database = 'Azure_EmployeeDB'

    # Connection string for ActiveDirectoryInteractive authentication
    # This will trigger a browser-based MFA prompt if needed
connection_string = (
        f'DRIVER={{ODBC Driver 18 for SQL Server}};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'Authentication=ActiveDirectoryInteractive;'
        f'UID=johan.robelto.bayona_outlook.com#EXT#@johanrobeltobayonaoutlook.onmicrosoft.com;'
        f'pwd=Outnano92;'
        )

try:
    # Establish the connection
    cnxn = pyodbc.connect(connection_string)
    print("Connection established successfully with MFA.")

    # Create a cursor object
    cursor = cnxn.cursor()

    # Define your SQL query
    query = "SELECT * FROM [SalesLT].[Customer];"  # Replace with your actual query

    # Execute the query and fetch results into a Pandas DataFrame
    df = pd.read_sql(query, cnxn)

    # Display the first few rows of the DataFrame
    print("\nQuery results:")
    print(df.head())

except pyodbc.Error as ex:
    sqlstate = ex.args[0]
    if sqlstate == 'FA004':
        print(f"Error connecting to SQL Server: {sqlstate}. Active Directory authentication failed.")
    else:
        print(f"An unexpected error occurred: {ex}")

finally:
    if 'cnxn' in locals() and cnxn:
        cnxn.close()

Error connecting to SQL Server: FA004. Active Directory authentication failed.


In [ ]:
import pypyodbc
import pandas as pd

SERVER_NAME = 'jarbdatabase.database.windows.net'
DATABASE_NAME = 'Azure_EmployeeDB'

conn = pypyodbc.connect("""
    Driver={{ODBC Driver 18 for SQL Server}};
    Server={0};
    Database={1};
    Trusted_Connection=yes;""".format(SERVER_NAME, DATABASE_NAME)
)

sql_query = """
SELECT TOP 5000 *
FROM [Table Name] WITH (NoLock)
"""

# With Headers
df1 = pd.read_sql(sql_query, conn)

# Without Headers
c = conn.cursor()
df2 = pd.DataFrame(c.execute(sql_query))

In [10]:
import pypyodbc as odbc
import pandas as pd
from credentials import username, password

server = 'jarbdatabase.database.windows.net'
database = 'Azure_EmployeeDB'
connection_string = 'DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+password
conn = odbc.connect(connection_string)

sql = '''
SELECT * FROM SalesLT.Customer
'''
cursor = conn.cursor()
cursor.execute(sql)
dataset = cursor.fetchall()

columns = [column [0] for column in cursor.description]
df = pd.DataFrame(dataset,columns=columns)
print(df)

Error: ('HY000', '[HY000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Cannot open server "johanrobeltobayonaoutlook.onmicrosoft.com" requested by the login.  The login failed.')

In [ ]:
import pypyodbc as odbc
import pandas as pd
from credentials import username, password

server = 'jarbdatabase.database.windows.net'
database = 'Azure_EmployeeDB'

# Toggle: set use_aad=True to use Azure AD interactive auth (opens interactive prompt)
# When using AAD interactive, DO NOT pass UID/PWD in the connection string.
use_aad = True

if use_aad:
	# Active Directory Interactive authentication (requires ODBC Driver 18+ and internet access)
	# The driver will open an interactive sign-in window for the user account.
	connection_string = (
		'DRIVER={ODBC Driver 18 for SQL Server};'
		f'SERVER={server};DATABASE={database};ENCRYPT=yes;'
		'Authentication=ActiveDirectoryInteractive;'
		# Optional: TrustServerCertificate=yes can help debugging TLS issues locally.
		# 'TrustServerCertificate=yes;'
	)
else:
	# Traditional SQL authentication using UID/PWD from credentials.py
	connection_string = (
		'DRIVER={ODBC Driver 18 for SQL Server};'
		f'SERVER={server};DATABASE={database};ENCRYPT=yes;'
		f'UID={username};PWD={password};'
	)

print('Using connection string:')
print(connection_string.replace(password, '***') if not use_aad else connection_string)

conn = odbc.connect(connection_string)

sql = '''
SELECT * FROM SalesLT.Customer
'''
cursor = conn.cursor()
cursor.execute(sql)
#dataset = cursor.fetchall()

#columns = [column[0] for column in cursor.description]
#df = pd.DataFrame(dataset, columns=columns)
#print(df)